In [ ]:
import pandas as pd
import math

# Load Excel dataset
df = pd.read_excel("cranfield_q1_dataset.xlsx")

# Filter only query q1
df_q1 = df[df["Query ID"] == "q1"].sort_values("Rank")
# Retrieved and relevant sets
retrieved = df_q1["Document ID"].tolist()
print(retrieved)
relevant = (df_q1[df_q1["Is_Relevant"] == "Yes"]["Document ID"].tolist())
print(relevant)

# Precision
def precision(retrieved, relevant):
    if not retrieved: return 0
    return sum(doc in relevant for doc in retrieved) / len(retrieved)

# Recall
def recall(retrieved, relevant):
    if not relevant: return 0
    return sum(doc in relevant for doc in retrieved) / len(relevant)

# F-measure
def f_measure(p, r):
    if (p + r) == 0: return 0
    return 2 * p * r / (p + r)

# E-measure
def e_measure(p, r, beta=1):
    if p == 0 and r == 0: return 1
    return 1 - ((1 + beta**2) * p * r) / (beta**2 * p + r)

# DCG
def dcg(retrieved, relevant):
    return sum(1 / math.log2(i+2) for i, doc in enumerate(retrieved) if doc in relevant)

# IDCG
def idcg(relevant, k):
    return sum(1 / math.log2(i+2) for i in range(min(len(relevant), k)))

# NDCG
def ndcg(retrieved, relevant):
    dcg_val = dcg(retrieved, relevant)
    idcg_val = idcg(relevant, len(retrieved))
    return dcg_val / idcg_val if idcg_val > 0 else 0

# ---- Run metrics ----
p = precision(retrieved, relevant)
r = recall(retrieved, relevant)
f = f_measure(p, r)
e = e_measure(p, r)
n = ndcg(retrieved, relevant)

print("Precision:", round(p, 3))
print("Recall:", round(r, 3))
print("F-measure:", round(f, 3))
print("E-measure:", round(e, 3))
print("NDCG:", round(n, 3))


['D184', 'D27', 'D373', 'D81', 'D276', 'D95', 'D144', 'D285', 'D322', 'D401']
['D184', 'D27', 'D276', 'D285', 'D401']
Precision: 0.5
Recall: 1.0
F-measure: 0.667
E-measure: 0.333
NDCG: 0.889


In [7]:
import pandas as pd
import math

# === Step 1: Load your real dataset ===
df = pd.read_excel("cranfield_q1_dataset.xlsx")

# === Step 2: Filter for Query q1 ===
df_q1 = df[df["Query ID"] == "q1"].sort_values("Rank")

# === Step 3: Extract Retrieved and Relevant Docs ===
retrieved = df_q1["Document ID"].tolist()
relevant = df_q1[df_q1["Is_Relevant"] == "Yes"]["Document ID"].tolist()

print("Retrieved Documents:", retrieved)
print("Relevant Documents:", relevant)

# === Step 4: Compute Precision and Recall ===
retrieved_relevant = [d for d in retrieved if d in relevant]
precision = len(retrieved_relevant) / len(retrieved) if retrieved else 0
recall = len(retrieved_relevant) / len(relevant) if relevant else 0

# === Step 5: F-measure ===
f_measure = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0

# === Step 6: E-measure (CORRECTED) ===
beta = 1
# Standard E-measure formula
if precision > 0 and recall > 0:
    e_measure = 1 - (1 / ((beta**2 / (1 + beta**2)) * (1/precision) + (1 / (1 + beta**2)) * (1/recall)))
else:
    e_measure = 1.0  # Maximum error when precision or recall is 0

# Alternative simpler E-measure (if beta=1):
# e_measure = 1 - f_measure

# === Step 7: NDCG Calculation ===
df_q1["rel_score"] = df_q1["Is_Relevant"].apply(lambda x: 1 if x == "Yes" else 0)

rels = df_q1["rel_score"].tolist()  # actual relevance scores list

# DCG (actual order)
dcg = sum(rels[i] / math.log2(i + 2) for i in range(len(rels)))

# IDCG (ideal order)
ideal_rels = sorted(rels, reverse=True)
idcg = sum(ideal_rels[i] / math.log2(i + 2) for i in range(len(ideal_rels)))
ndcg = dcg / idcg if idcg > 0 else 0

# === Step 8: Display Results ===
print("\n=== Information Retrieval Metrics for Query q1 ===")
print("Precision :",round(precision, 3))
print(f"Recall    : {recall:.3f}")
print(f"F-measure : {f_measure:.3f}")
print(f"E-measure : {e_measure:.3f}")
print(f"NDCG      : {ndcg:.3f}")

Retrieved Documents: ['D184', 'D27', 'D373', 'D81', 'D276', 'D95', 'D144', 'D285', 'D322', 'D401']
Relevant Documents: ['D184', 'D27', 'D276', 'D285', 'D401']

=== Information Retrieval Metrics for Query q1 ===
Precision : 0.5
Recall    : 1.000
F-measure : 0.667
E-measure : 0.333
NDCG      : 0.889
